In [ ]:
import codecs
from datetime import datetime as dt
import json
import sys
import numpy as np
import os
import pandas as pd
import plotly
from plotly import subplots
import plotly.express as px
import plotly.tools as tls
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as offline
import sys
if "ipy" in sys.argv[0]:
    offline.init_notebook_mode()
from cov19utils import create_basic_plot_figure,     show_and_clear, moving_average,     blank2zero, csv2array,     get_twitter, tweet_with_image,     get_gpr_predict, FONT_NAME, DT_OFFSET,     download_if_needed, json2nparr, code2int, age2int,     get_populations, get_os_idx_of_arr, dump_val_in_arr,     calc_last1w2w_dif, create_basic_scatter_figure,     show_and_save_plotly
import re
import requests

In [ ]:
# https://twitter.com/zwiYWJqourMuEh7/status/1431351917330845696

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import ConstantKernel as C
from sklearn.gaussian_process.kernels import WhiteKernel as WK
from sklearn.gaussian_process.kernels import RationalQuadratic as RQ

In [ ]:
from scipy.optimize import curve_fit
from scipy.special import factorial

In [ ]:
df = pd.read_csv('toma-ih.csv')
fromto = "{}～{}".format(df.iloc[0]['Date'], df.iloc[-1]['Date'])
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')

In [ ]:
title = '全国高校選抜アイスホッケー大会COVID-19感染者数'

In [ ]:
df['x'] = 0
df['x'] = df['Date'] - df.iloc[0]['Date']
df['x'] = df['x'].map(lambda x: x.days)

In [ ]:
y = df['Cases'].tolist()
X = df['x'].to_numpy().reshape(-1, 1)
X_pred = df['x'].tolist()
l = X_pred[-1]
for i in np.arange(7):
    X_pred.append(l + i + 1)
X_pred = np.array(X_pred).reshape(-1, 1)

In [ ]:
kernel = 1e-1**2 * RBF(length_scale=1) + WK(noise_level=1e-1**2)

In [ ]:
gp = GaussianProcessRegressor(
    kernel=kernel,
    alpha=1e-1, 
    optimizer="fmin_l_bfgs_b",
    normalize_y=True,
    n_restarts_optimizer=10
)

In [ ]:
gp.fit(X, y)
y_pred, sigma = gp.predict(X_pred, return_std=True)

In [ ]:
def pred_func(x, b, c, K):
    return K / (1 + b * np.exp(-c * x))

In [ ]:
(b, c, K), p0 = curve_fit(
    pred_func, df['x'].to_numpy(), y, maxfev=1000,
    p0=[3, 1, 300]
    )
print("b:{}, c:{}, K:{}, p0:{}".format(b, c, K, p0))

In [ ]:
yhat = pred_func(X_pred.reshape(1, -1)[0], b, c, K)

In [ ]:
fig = go.Figure()
obs = go.Scatter(
    name='Observations', mode='markers', marker=dict(color='red'),
    x=X.reshape(1, -1)[0], y=df['Cases'])
pred = go.Scatter(
    name='Prediction', mode='lines', line=dict(color='blue'),
    x=X_pred.reshape(1, -1)[0], y=y_pred)
conf = go.Scatter(
    x=np.concatenate([X_pred, X_pred[::-1]]).reshape(1, -1)[0],
    y=np.concatenate([y_pred - 0.10 * sigma,
                      (y_pred + 1.9600 * sigma)[::-1]]),
    mode='lines',
    line=dict(color='blue', width=0.5),
    opacity=0.6,
    fill='toself',
    name='95% confidence interval'
)
pois = go.Scatter(
    x=X_pred.reshape(1, -1)[0], y=yhat,
    name='Logistic'
)
fig.add_trace(obs)
fig.add_trace(conf)
fig.add_trace(pred)
fig.add_trace(pois)
fig.update_layout(
    width=800, height=600,
    margin={"r":20,"t":50,"l":20,"b":50},
    title=title,
    template='plotly_dark', xaxis_title='日付',
    yaxis_title='人数',
)
show_and_save_plotly(
    fig, "tom-ih.jpg", js=False, show=True, image=True, html=False)

print("{} {}".format(title, fromto))